<img src="https://raw.githubusercontent.com/codecaviar/digital_asset_management/master/assets/bingyune-and-company-logo-6400x3600.png" align="left" width="200" height="auto">

<br/><br/><br/><br/>

# Bike Share for All: Beginner's Guide to Basic SQL

**BingYune Chen**, Principal Data Scientist<br>
2020-07-30 | 12 minute read

Data is provided by [Kaggle](https://www.kaggle.com/benhamner/sf-bay-area-bike-share) 
    | Source code is on [Github](https://github.com/codecaviar)

---

The goal of this project is to provide a Beginner's Guide to the basic skills needed to use SQL. The SQL programming language is a standard of the American National Standards Institute (ANSI).

## Table of Contents

1. [**Project Overview**](#overview)
2. [**Using SQL in Jupyter Notebook**](#sqlite_jupyter)
3. [**Basic SQL**](#basic_sql)
    1. [**SQL SELECT and FROM**](#sql_select_from)
    2. [**SQL WHERE**](#sql_where)
    3. [**SQL ORDER BY**](#sql_orderby)
    4. [**SQL LIMIT**](#sql_limit)
4. [**Bonus: Comments in SQL**](#bonus)
5. [**Conclusion**](#conclusion)

<a class="anchor" id="overview"></a>
# 1. Project Overview

Structured Query Language, or more commonly known as SQL (pronounced "ess-cue-ell" or "si-kwel"), is a standard programming language for accessing and manipulating databases. Though SQL is commonly used by engineers in software development, SQL is also popular with data scientists and analysts because of it's ability to scale. Traditional spreadsheets can be used to manage small-to-medium-sized pools of data such as between multiple worksheets, but you will need a different solution when handling excessively large records. Whether it's 1,000 records or 100 million, SQL can rapidly navigate databases and query, retrieve, and aggregate records. SQL is also more adept than spreadsheets at creating data flows for cleaning and preparing data at high volumes. Because SQL allows users to also remotely interact with large datasets in production environments, SQL is still the industry standard in data science and analytics for data query and retrieval. Almost all of the biggest names in tech use SQL - Netflix, Instagram, LinkedIn, Lyft, Dropbox - the list goes on. Long story short: yes, you need to learn SQL to be more qualified for a job in data.

<a class="anchor" id="problem_statement"></a>
## 1.1 Problem Statement

The goal of this project is to provide a Beginner's Guide to the basic hacking skills needed to start analyzing data with SQL. The project makes use of the [Bay Area Bike Share](https://mtc.ca.gov/our-work/operate-coordinate/traveler-services/bay-area-bike-share) data sourced from [Kaggle](https://www.kaggle.com/benhamner/sf-bay-area-bike-share). The Bay Area Bike Share program enables quick, easy, and affordable bike trips around the San Francisco Bay Area. The original dataset contains data for 70 stations (where users can pickup or return bikes), about 71M status updates (number of bikes and docks available for given station), about 670k trips (individual bike trips), and about 3k weather forecasts (for a specific day for a certain zip code). 

When it comes to SQL databases, transactional tables are large and updated frequently, whereas reference tables are smaller and rarely modified. The `station` table used in this guide is a reference table, as each row represents a single bike station with associated information for that bike station. 

Here is some additional information about the `station` table:
* `id` is a primary key (unique identifier for each row)
* `name` is the official name of each bike station
* `lat` is the latitude coordinates of the bike station
* `long` is the longitude coordinates of the bike station
* `dock_count` is the count of the number of bike docks at the bike station
* `city` is the name of the city where the bike station is located
* `installation_date` is the date the bike station was installed

<a class="anchor" id="sqlite_jupyter"></a>
# 2. Using SQL in Jupyter Notebook

The entire guide is meant to be completed using Juptyer Notebook, combined with the SQLite and Pandas libraries. You'll retain the most information if you run the example queries and try to understand results, and complete the practice exercises.

[Jupyter Notebook](https://jupyter.org/) is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, data visualization, machine learning, and much more.

[SQLite](https://www.sqlite.org/about.html) is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite database files are commonly used as containers to transfer rich content between systems and as a long-term archival format for data.

[Pandas](https://pandas.pydata.org/about/) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language. The library contains a fast and efficient DataFrame object for data manipulation with integrated indexing.

The function `sqlpd` is the bread and butter of how we will be running SQL code in Jupyter Notebook - it's where you'll be able to use all the skills you learn in this guide. This function connects to the SQLite database, reads the raw SQL code, and outputs the results as a pandas DataFrame.

In [1]:
# Add your file path to the downloaded database.sqlite file from Kaggle
FILE_PATH = '../bike_share_sql_data/database.sqlite' # default value

# Set up a database using SQLite and Pandas
import pandas as pd
import sqlite3

# Create function to read SQL query statements
def sqlpd(sql_query):
    """
    Read a string as SQL statement to query a SQLite database
    and output a pandas dataframe of the resulting query
    
    Args:
        sql_query (string): raw SQL code
        
    Returns:
        df (pd.DataFrame): two-dimensional, size-mutable, heterogeneous tabular data
    """
    # Connect to SQLite database
    sqlite_file = FILE_PATH 
    con = sqlite3.connect(sqlite_file) # interface for SQLite database
    
    # Use pandas to pass sql query using connection from SQLite3
    df = pd.read_sql_query(sql_query, con)
    
    # Close the SQLite database
    con.close()

    # Show the resulting DataFrame
    return df

For this guide, SQL queries will be shown in each Jupyter Notebook cell with the string variable `query` and three double quotation marks ( `"""` ) like the one below. To run a query, simply change the text between the three double quotation marks ( `"""` ). Run the Jupyter Notebook cell (shift + enter). It might take a few seconds for the query to run. When it's done, you'll see the results show up in a pandas DataFrame table below the query window. Note that the DataFrame table isn't stored in the Python [namespace](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces) or permanently in the SQLite database. The `query` also doesn't change any tables in the database - `database.sqlite` will contain the same data every time you query it, and the data will never change no matter how many times you query it.

In [2]:
# Check connection to database with a SQL query for all tables
query = """
SELECT name 
FROM sqlite_master
WHERE type='table'
;
"""

sqlpd(query) # returns all table names in the database

,name
0,station
1,status
2,trip
3,weather


If you've used spreadsheets, you should already be familiar with SQL tables (similar to a worksheet with rows and columns). Multiple tables can be organized by "schemas" (or roadmap) in a single SQL database on SQL servers. A schema is a roadmap that represents a database's structure, as well as the relationships between its tables. The schema also contains the rules that dictate database behavior, such as what columns exist in each table, each column's data type (e.g. TEXT and INT), and how each of the tables relate to one another.

Every table is broken up into smaller entities called fields. For instance, the fields in the "station" table consist of id, name, lat, long, dock_count, city, and installation_date. A field is a column in a table that is designed to maintain specific information about every record in the table.

A record, also called a row, is each individual entry that exists in a table. For example, there are 70 records in the "station" table. A record is a horizontal entity in a table.

A column is a vertical entity in a table that contains all information associated with a specific field in a table.

To understand our data, we need to know what's in the stored data. One way to do this is by telling the story of one row: Read across one row of your data to really see and understand the values contained in every column. This will help you get to know what's in your data set or propel you to investigate further if you don't understand a particular column.

# 3. Basic SQL

Although there are a number of different types of SQL database implementations and platforms (e.g. SQLite - this guide uses this one, PostgreSQL, MySQL, Microsoft SQL Server, and Oracle Database), all versions support at least the major SQL clauses (such as `SELECT`, `FROM`, and `WHERE`) in a similar fashion. 

* **SELECT**: *selects* the columns
* **FROM**: *points* to the table
* **WHERE**: *filters* on rows
* **GROUP BY**: *aggregates* across values of a variable
* **HAVING**: *filters* groups
* **ORDER BY**: *sorts* or *arranges* the results
* **LIMIT**: *limits* the results to the first n rows

You might have noticed that the `SELECT` and other SQL clauses above are capitalized. This capitalization makes the code easier to read, but it doesn't actually affect the code's function in any way. If you prefer to write your queries with lowercase clauses, the queries will still execute correctly. Similarly, SQL reads one space, multiple spaces, or a line break as being the same thing. The best practice is to end each SQL statement with a semicolon `;`, especially when more than one SQL statement is to be executed in the same call to the server. While we're on the topic of formatting, it's also worth noting the format of column names. All of the columns in the `database.sqlite` database from Kaggle are named in lower case, and use underscores instead of spaces. 

<a class="anchor" id="sql_select_from"></a>
## 3.1 SQL SELECT and FROM

`SELECT` allows you to select certain columns from a table and determines which columns of information are downloaded.

`FROM` specifies the tables from which the query extracts data and defines the relationships between the tables.

So what's happening in the below query? In this case, the query is telling the database to return `name`, `dock_count`, and `city` columns from the table `station` in the database `database.sqlite`. When you run this query, you'll get back a set of results that shows values in each of these columns. Note that whenever you select multiple columns, they must be separated by commas, but you should **not** include a comma after the last column name.

Alternatively, using the `*` instead of specific column names will select every column in a table.

In [3]:
# Run SQL statement
query = """
SELECT name, dock_count, city
FROM station
;
"""

sqlpd(query) # returns query result as pandas DataFrame

,name,dock_count,city
0,San Jose Diridon Caltrain Station,27,San Jose
1,San Jose Civic Center,15,San Jose
2,Santa Clara at Almaden,11,San Jose
3,Adobe on Almaden,19,San Jose
4,San Pedro Square,15,San Jose
...,...,...,...
65,Market at Sansome,27,San Francisco
66,Santa Clara County Civic Center,15,San Jose
67,Broadway St at Battery St,15,San Francisco
68,Mezes Park,15,Redwood City


In [4]:
# Run SQL statement using * 
query = """
SELECT *
FROM station
;
"""

sqlpd(query) # returns query result as pandas DataFrame

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


In [5]:
# Practice Problem
# Write a query to select all of the columns in the 'station' table without using *

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


In [6]:
# Add DISTINCT to the SELECT statement to eliminate duplicates
query = """
SELECT DISTINCT city
FROM station
;
"""

sqlpd(query) # returns table of unique cities with bike stations 

# If you include two (or more) columns in a SELECT DISTINCT clause
# your results will contain all of the unique pairs of those two columns.

,city
0,San Jose
1,Redwood City
2,Mountain View
3,Palo Alto
4,San Francisco


In [109]:
# Practice Problem
# Write a query to get the distinct count of docks across all stations

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,dock_count
0,27
1,15
2,11
3,19
4,25
5,23


<a class="anchor" id="sql_where"></a>
## 3.2 SQL WHERE

`WHERE` filters and focuses resulting information and can be used to filter different types of data. 

So what's happening in the below query? In this case, the query is telling the database to return `*` columns from the table `station` in the database `database.sqlite`, where the value for `city` is equal to "San Francisco". When you run this query, you'll get back a set of results that shows values in each of these columns with an applied filter. Note the column `city` only contains the values of "San Francisco" because of the applied filter. If you write a `WHERE` clause that filters based on values in one column, you'll limit the results in all columns to rows that satisfy the condition. The idea is that each row is one data point or observation, and all the information contained in that row belongs together.

In [7]:
# Which stations are in the city of San Francisco?
query = """
SELECT *
FROM station
WHERE city = 'San Francisco'
;
"""

sqlpd(query)

,id,name,lat,long,dock_count,city,installation_date
0,39,Powell Street BART,37.783871,-122.408433,19,San Francisco,8/25/2013
1,41,Clay at Battery,37.795001,-122.399970,15,San Francisco,8/19/2013
2,42,Davis at Jackson,37.797280,-122.398436,15,San Francisco,8/19/2013
3,45,Commercial at Montgomery,37.794231,-122.402923,15,San Francisco,8/19/2013
4,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,8/19/2013
5,47,Post at Kearney,37.788975,-122.403452,19,San Francisco,8/19/2013
6,48,Embarcadero at Vallejo,37.799953,-122.398525,15,San Francisco,8/19/2013
7,49,Spear at Folsom,37.790302,-122.390637,19,San Francisco,8/20/2013
8,50,Harry Bridges Plaza (Ferry Building),37.795392,-122.394203,23,San Francisco,8/20/2013
9,51,Embarcadero at Folsom,37.791464,-122.391034,19,San Francisco,8/20/2013


In [8]:
# Practice Problem
# Select the station with the name "Powell Street BART"

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,id,name,lat,long,dock_count,city,installation_date
0,39,Powell Street BART,37.783871,-122.408433,19,San Francisco,8/25/2013


You can also filter your results in a number of ways using comparison and logical operators. The most basic way to filter data is using **comparison operators**. Think back to your elementary school days of arithmetic and math equations:

* Equal to ... = or ==
* Not equal to ... != or <>
* Greater than ... >
* Greater than or equal to ... >=
* Less than ... <
* Less than or equal to ... <=

Similar to math equations, the order of operations does matter. Parentheses go first and can be used to create groupings or priorities. 

In [9]:
# Which stations are not in the city of San Jose?
query = """
SELECT *
FROM station
WHERE city != 'San Jose'
;
"""

sqlpd(query)

,id,name,lat,long,dock_count,city,installation_date
0,21,Franklin at Maple,37.481758,-122.226904,15,Redwood City,8/12/2013
1,22,Redwood City Caltrain Station,37.486078,-122.232089,25,Redwood City,8/15/2013
2,23,San Mateo County Center,37.487616,-122.229951,15,Redwood City,8/15/2013
3,24,Redwood City Public Library,37.484219,-122.227424,15,Redwood City,8/12/2013
4,25,Stanford in Redwood City,37.485370,-122.203288,15,Redwood City,8/12/2013
5,26,Redwood City Medical Center,37.487682,-122.223492,15,Redwood City,8/12/2013
6,27,Mountain View City Hall,37.389218,-122.081896,15,Mountain View,8/16/2013
7,28,Mountain View Caltrain Station,37.394358,-122.076713,23,Mountain View,8/15/2013
8,29,San Antonio Caltrain Station,37.406940,-122.106758,23,Mountain View,8/15/2013
9,30,Evelyn Park and Ride,37.390277,-122.066553,15,Mountain View,8/16/2013


In [10]:
# Which stations have less than 15 docks?
query = """
SELECT *
FROM station
WHERE dock_count < 15
;
"""

sqlpd(query) # all stations have more than 10 docks

,id,name,lat,long,dock_count,city,installation_date
0,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
1,32,Castro Street and El Camino Real,37.385956,-122.083678,11,Mountain View,12/31/2013
2,35,University and Emerson,37.444521,-122.163093,11,Palo Alto,8/15/2013
3,37,Cowper at University,37.448598,-122.159504,11,Palo Alto,8/14/2013


In [11]:
# Practice Problem
# Which stations have at least 20 docks?

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,22,Redwood City Caltrain Station,37.486078,-122.232089,25,Redwood City,8/15/2013
2,28,Mountain View Caltrain Station,37.394358,-122.076713,23,Mountain View,8/15/2013
3,29,San Antonio Caltrain Station,37.406940,-122.106758,23,Mountain View,8/15/2013
4,34,Palo Alto Caltrain Station,37.443988,-122.164759,23,Palo Alto,8/14/2013
5,50,Harry Bridges Plaza (Ferry Building),37.795392,-122.394203,23,San Francisco,8/20/2013
6,55,Temporary Transbay Terminal (Howard at Beale),37.789756,-122.394643,23,San Francisco,8/20/2013
7,59,Golden Gate at Polk,37.781332,-122.418603,23,San Francisco,8/21/2013
8,61,2nd at Townsend,37.780526,-122.390288,27,San Francisco,8/22/2013
9,67,Market at 10th,37.776619,-122.417385,27,San Francisco,8/23/2013


The comparison operators also work for non-numerical data columns. If you're using an operator (e.g. = or !=) with values that are non-numeric, you need to put the value in single or double quotes: 'value' (e.g. 'San Jose' from an earlier query). Note it's less common to use >, <, >=, or <= operators with values that are non-numeric because SQL treats alphabetical ordering in a possibly unintuitive manner (more like an English dictionary, less like counting numbers).

Unlike comparison operators, **logical operators** allow you to use multiple comparison operators in one query. These operators work especially well for non-numerical data.

* `LIKE` ... pattern matching for similar values using % and -
* `IN` ... matches a value in a list
* `BETWEEN` ... select only rows within a range (inclusive)
* `IS NULL` ... select rows that contain no data in a given column
* `AND` ... select only rows that satisfy two conditions
* `OR` ... select rows that satisfy either of two conditions
* `NOT` ... negates a condition

In [12]:
# LIKE

# Which stations have the word "BART" in the name?
query = """
SELECT name
FROM station
WHERE name LIKE '%BART%'  
;
"""

sqlpd(query)

,name
0,Powell Street BART
1,Civic Center BART (7th at Market)


The `%` (a percent sign) used above represents any character or set of characters. In this case, `%` is referred to as a "wildcard." The SQL statement above will include all of the characters before "BART" (including blank spaces and symbols) and any number of characters after "BART" (including blank spaces and symbols). In the type of SQL that SQLite uses, LIKE is case-sensitive, meaning that the above query will only capture matches that use all capitalization for "BART." To ignore case when you're matching values, you can use the ILIKE operator. You can also use `_` (a single underscore) to substitute for an individual character.

In [13]:
# Practice Problem
# Which stations have the word "University" in the name?

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,name
0,University and Emerson
1,Cowper at University


In [14]:
# NOT LIKE

# Which stations do not have the word "BART" in the name?
query = """
SELECT name
FROM station
WHERE name NOT LIKE '%BART%'  
;
"""

sqlpd(query)

# In some cases, the LIKE and NOT LIKE queries may not total
# the full amount of rows for a table because of null values

,name
0,San Jose Diridon Caltrain Station
1,San Jose Civic Center
2,Santa Clara at Almaden
3,Adobe on Almaden
4,San Pedro Square
...,...
63,Market at Sansome
64,Santa Clara County Civic Center
65,Broadway St at Battery St
66,Mezes Park


In [15]:
# IN

# Which stations are either in the city of San Francisco 
# or in the city of Mountain View?
query = """
SELECT name, city
FROM station
WHERE city IN ('San Francisco', 'Mountain View')  
;
"""

sqlpd(query)

,name,city
0,Mountain View City Hall,Mountain View
1,Mountain View Caltrain Station,Mountain View
2,San Antonio Caltrain Station,Mountain View
3,Evelyn Park and Ride,Mountain View
4,San Antonio Shopping Center,Mountain View
5,Castro Street and El Camino Real,Mountain View
6,Rengstorff Avenue / California Street,Mountain View
7,Powell Street BART,San Francisco
8,Clay at Battery,San Francisco
9,Davis at Jackson,San Francisco


In [16]:
# Practice Problem
# Which stations are not in the city of San Francisco or the city of Mountain View?

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,name,city
0,San Jose Diridon Caltrain Station,San Jose
1,San Jose Civic Center,San Jose
2,Santa Clara at Almaden,San Jose
3,Adobe on Almaden,San Jose
4,San Pedro Square,San Jose
5,Paseo de San Antonio,San Jose
6,San Salvador at 1st,San Jose
7,Japantown,San Jose
8,San Jose City Hall,San Jose
9,MLK Library,San Jose


In [17]:
# BETWEEN

# Which stations have between 15 and 20 docks?
query = """
SELECT name, dock_count
FROM station
WHERE dock_count BETWEEN 15 AND 20
;
"""

sqlpd(query)

,name,dock_count
0,San Jose Civic Center,15
1,Adobe on Almaden,19
2,San Pedro Square,15
3,Paseo de San Antonio,15
4,San Salvador at 1st,15
5,Japantown,15
6,San Jose City Hall,15
7,MLK Library,19
8,SJSU 4th at San Carlos,19
9,St James Park,15


In [18]:
# AND plus OR

# How do I find all stations in the city of San Francisco or 
# any stations in the city of Palo Alto with more than 25 docks?

query = """
SELECT *
FROM station
WHERE city = 'San Francisco' 
    OR (city = 'Palo Alto' AND dock_count >= 20)
;
"""

sqlpd(query)

,id,name,lat,long,dock_count,city,installation_date
0,34,Palo Alto Caltrain Station,37.443988,-122.164759,23,Palo Alto,8/14/2013
1,39,Powell Street BART,37.783871,-122.408433,19,San Francisco,8/25/2013
2,41,Clay at Battery,37.795001,-122.399970,15,San Francisco,8/19/2013
3,42,Davis at Jackson,37.797280,-122.398436,15,San Francisco,8/19/2013
4,45,Commercial at Montgomery,37.794231,-122.402923,15,San Francisco,8/19/2013
5,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,8/19/2013
6,47,Post at Kearney,37.788975,-122.403452,19,San Francisco,8/19/2013
7,48,Embarcadero at Vallejo,37.799953,-122.398525,15,San Francisco,8/19/2013
8,49,Spear at Folsom,37.790302,-122.390637,19,San Francisco,8/20/2013
9,50,Harry Bridges Plaza (Ferry Building),37.795392,-122.394203,23,San Francisco,8/20/2013


In [19]:
# Practice Problem
# How do you find all stations with the word "BART" in the name 
# or any stations in the city of San Francisco with more than 25 docks?

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,id,name,lat,long,dock_count,city,installation_date
0,39,Powell Street BART,37.783871,-122.408433,19,San Francisco,8/25/2013
1,61,2nd at Townsend,37.780526,-122.390288,27,San Francisco,8/22/2013
2,67,Market at 10th,37.776619,-122.417385,27,San Francisco,8/23/2013
3,72,Civic Center BART (7th at Market),37.781039,-122.411748,23,San Francisco,8/23/2013
4,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013


<a class="anchor" id="sql_orderby"></a>
## 3.3 SQL ORDER BY

`ORDER BY` sorts results in ascending or descending order. After `ORDER BY` is a number that indicates the column by which you're sorting or the name of the column. 

The default sort order is ascending (first to last or smaller to larger), but you can specify ascending (`ASC`) or descending (`DESC`) to determine the sort order.

So what's happening in the below query? In this case, the query is telling the database to return `*` columns from the table `station` in the database `database.sqlite` and order the rows by the `name` column. When you run this query, you'll get back a set of results that shows values for all columns, but sorted in order by the name of the station. You'll notice that the results are now ordered alphabetically from a to z based on the content in the `name` column. 

In [20]:
# Select all stations in alphabetical order by name
query = """
SELECT *
FROM station
ORDER BY name
;
"""

sqlpd(query)

,id,name,lat,long,dock_count,city,installation_date
0,62,2nd at Folsom,37.785299,-122.396236,19,San Francisco,8/22/2013
1,64,2nd at South Park,37.782259,-122.392738,15,San Francisco,8/22/2013
2,61,2nd at Townsend,37.780526,-122.390288,27,San Francisco,8/22/2013
3,57,5th at Howard,37.781752,-122.405127,15,San Francisco,8/21/2013
4,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
...,...,...,...,...,...,...,...
65,55,Temporary Transbay Terminal (Howard at Beale),37.789756,-122.394643,23,San Francisco,8/20/2013
66,65,Townsend at 7th,37.771058,-122.402717,15,San Francisco,8/22/2013
67,35,University and Emerson,37.444521,-122.163093,11,Palo Alto,8/15/2013
68,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,8/19/2013


You can also order by multiple columns. This is particularly useful if your data falls into categories and you'd like to organize rows one column, but keep all of the results within a given category together.

So what's happening in the below query? In this case, the query is telling the database to return `*` columns from the table station in the database `database.sqlite`, where `dock_count` is greater than 20, sorted in descending order by `city`, THEN sorted in ascending order by `dock_count`. When you run this query, you'll get back a set of results that shows values for all columns, with more than 20 docks at the station, sorted in order by city and the number of docks.

In [21]:
# Select all stations in order by city and dock count
query = """
SELECT *
FROM station
WHERE dock_count < 20
ORDER BY city DESC, dock_count
;
"""

sqlpd(query)

,id,name,lat,long,dock_count,city,installation_date
0,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
3,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
4,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,8/5/2013
5,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
6,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
7,13,St James Park,37.339301,-121.889937,15,San Jose,8/6/2013
8,16,SJSU - San Salvador at 9th,37.333955,-121.877349,15,San Jose,8/7/2013
9,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013


In [22]:
# Practice Problem
# Select all stations in the city of San Jose, ordered by station name and dock count

# Try it out
try:
    query = """
    ???
    """
    sqlpd(query)

except:
    None
    
# See the answer in the solution notebook

,id,name,lat,long,dock_count,city,installation_date
0,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
1,14,Arena Green / SAP Center,37.332692,-121.900084,19,San Jose,8/5/2013
2,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
3,11,MLK Library,37.335885,-121.885660,19,San Jose,8/6/2013
4,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
5,84,Ryland Park,37.342725,-121.895617,15,San Jose,4/9/2014
6,16,SJSU - San Salvador at 9th,37.333955,-121.877349,15,San Jose,8/7/2013
7,12,SJSU 4th at San Carlos,37.332808,-121.883891,19,San Jose,8/7/2013
8,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
9,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013


<a class="anchor" id="sql_limit"></a>
## 3.4 SQL LIMIT

`LIMIT` restricts how many rows the SQL query returns.

You can use `LIMIT` as a simple way to keep SQL queries from taking too long to return. When you first start exploring a dataset, the aim of many of your queries will simply be to see what a particular table looks like (get an idea of the general shape and column values). For example, you may just want to scan the first few rows of data to get an idea of which fields are relevant for your analysis. If you query a very large table (such as one with millions of rows) and don't use `LIMIT`, you could end up waiting a long time for all of your results to be displayed, which doesn't make sense if you only care about the first few. 

In [23]:
# Select all stations, but return only the first 5 results
query = """
SELECT *
FROM station
LIMIT 5
;
"""

sqlpd(query)

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


<a class="anchor" id="bonus"></a>
# 4. Bonus: Comments in SQL

We’ll briefly walk through three types of comments you are likely to encounter:

* **Header Comments**: /* comment across multiple lines */ 
* **In-Line Comments**: --comment
* **Multi-Line Comments**: /* comment, new line */

**Header comments** are a great way to keep a history of why a SQL query was built and who requested any changes that were made. When commenting in the header of a file, we recommend following a standardized format, like this one:

```
/**********************
NAME: Name of report.
DESC: Description of report.
AUTH: Name of author.
DATE: Date report published.
***********************
Change History
***********************
Version | Date | Author | Description
***********************/
```

**In-line comments** are usually used to explain calculations.

```
SELECT name --This is an in-line comment about this query.
```

**Multi-line comments** are a good way to document more complicated processes and the reasoning behind them, but should be used sparingly.

```
SELECT name, installation_date /* Here is an example of a 
multi-line comment about this query. */
```

<a class="anchor" id="conclusion"></a>
# 5. Conclusion

SQL can query, retrieve, and aggregate millions of records. SQL is
cloud-based data query and retrieval is not limited to your local computer system. SQL can organize data tables. SQL allows users to remotely interact with large data sets in production environments. Although SQL has quite a few impressive superpowers, there is also one main limitation...SQL is not a data visualization tool. It's normally used in conjunction with other tools such as Python, Tableau, Excel.

Congrats on completing the Beginner's Guide to Basic SQL!

You may find that your skills are still limited. If you want to do things like aggregate data across entire columns or merge multiple datasets together, check out the [Complete Guide to Intermediate SQL](https://www.bingyune.com/blog/bike-share-all-intermediate-sql).

<a class="anchor" id="acknowledgments"></a>
### Acknowledgments 

This project was inspired by bootcamps from General Assembly.

[General Assembly](https://generalassemb.ly/education/data-science-immersive/san-francisco) is a global education company on a mission to empower individuals and companies through dynamic training programs, exclusive thought leader events, and high-impact networking opportunities. The curricula focus on the in-demand skills every company today needs: coding, data, design, digital marketing, and product management. General Assembly also works with companies, from startups to more than 40 of the Fortune 100, to provide innovative tech training, onboarding, and hiring strategies to solve talent gaps.

The project referenced the following resources:
* https://mode.com/sql-tutorial/
* https://www.w3schools.com/sql/
* https://www.sqlitetutorial.net/

---
<em>The Code Caviar</em> is a digital magazine about data science and analytics that dives deep into key topics, so you can experience the thrill of solving at scale. 